In [2]:
from glob import glob
from helpers import create_polling_id
import json
import os
import pandas as pd
import re

os.chdir("../")

In [1]:
election_type = "camera"
year = 1987
# https://elezionistorico.interno.gov.it/eligendohome/opendata.php
# https://www.lemonde.fr/en/international/article/2022/09/24/italian-elections-mixed-voting-block-voting-and-multiple-candidacies-how-does-the-electoral-system-work_5998104_4.html

In [3]:
pd.read_csv(f"data/Italy/{election_type}-{year}.zip", delimiter=";", encoding="latin-1")

,CIRCOSCRIZIONE,PROVINCIA,COMUNE,ELETTORI,ELETTORI_MASCHI,VOTANTI,VOTANTI_MASCHI,SCHEDE_BIANCHE,LISTA,VOTI_LISTA
0,MANTOVA-CREMONA,CREMONA,PIERANICA,600,NaN,583,NaN,16,ALL.POP.,0
1,MANTOVA-CREMONA,CREMONA,PIERANICA,600,NaN,583,NaN,16,PSDI,7
2,MANTOVA-CREMONA,CREMONA,PIERANICA,600,NaN,583,NaN,16,PS D'AZ.,1
3,MANTOVA-CREMONA,CREMONA,PIERANICA,600,NaN,583,NaN,16,LIGA VENETA-PU,10
4,MANTOVA-CREMONA,CREMONA,PIERANICA,600,NaN,583,NaN,16,PSI,90
...,...,...,...,...,...,...,...,...,...,...
117340,PALERMO-TRAPANI-AGRIGENTO-CALTANISSETTA,AGRIGENTO,SICULIANA,5474,NaN,3326,NaN,64,LISTA VERDE,6
117341,PALERMO-TRAPANI-AGRIGENTO-CALTANISSETTA,AGRIGENTO,SICULIANA,5474,NaN,3326,NaN,64,MSI-DN,70
117342,PALERMO-TRAPANI-AGRIGENTO-CALTANISSETTA,AGRIGENTO,SICULIANA,5474,NaN,3326,NaN,64,PRI,8
117343,PALERMO-TRAPANI-AGRIGENTO-CALTANISSETTA,AGRIGENTO,SICULIANA,5474,NaN,3326,NaN,64,P.RAD,16


In [3]:
import zipfile
import pandas as pd
with zipfile.ZipFile(f"data/Italy/{election_type}-{year}.zip") as z:
    with z.open("Camera_Italia_LivComune.txt") as f:
        df = pd.read_csv(f, header=0, delimiter=";")

df["candidate"] = df["NOME"] + " " + df["COGNOME"]
df["polling_id"] = create_polling_id(df, columns=["CODTIPOELEZIONE", "CIRC-REG", "COLLPLURI", "COLLUNINOM", "COMUNE"])
df.head()

,DATAELEZIONE,CODTIPOELEZIONE,CIRC-REG,COLLPLURI,COLLUNINOM,COMUNE,ELETTORITOT,ELETTORIM,VOTANTITOT,VOTANTIM,...,VOTILISTA,DESCRLISTA,COGNOME,NOME,LUOGONASCITA,DATANASCITA,SESSO,VOTICANDIDATO,candidate,polling_id
0,25/9/2022 00:00:00,C,PIEMONTE 1,PIEMONTE 1 - P01,PIEMONTE 1 - U03 (COLLEGNO),CASELLE TORINESE,10851,5312,7075,3523,...,631,LEGA PER SALVINI PREMIER,MACCANTI,ELENA,TORINO,05/02/1971,F,2892,ELENA MACCANTI,C-PIEMONTE 1-PIEMONTE 1 - P01-PIEMONTE 1 - U03...
1,25/9/2022 00:00:00,C,PIEMONTE 1,PIEMONTE 1 - P01,PIEMONTE 1 - U03 (COLLEGNO),CASELLE TORINESE,10851,5312,7075,3523,...,476,FORZA ITALIA,MACCANTI,ELENA,TORINO,05/02/1971,F,2892,ELENA MACCANTI,C-PIEMONTE 1-PIEMONTE 1 - P01-PIEMONTE 1 - U03...
2,25/9/2022 00:00:00,C,PIEMONTE 1,PIEMONTE 1 - P01,PIEMONTE 1 - U03 (COLLEGNO),CASELLE TORINESE,10851,5312,7075,3523,...,23,NOI MODERATI/LUPI - TOTI - BRUGNARO - UDC,MACCANTI,ELENA,TORINO,05/02/1971,F,2892,ELENA MACCANTI,C-PIEMONTE 1-PIEMONTE 1 - P01-PIEMONTE 1 - U03...
3,25/9/2022 00:00:00,C,PIEMONTE 1,PIEMONTE 1 - P01,PIEMONTE 1 - U03 (COLLEGNO),CASELLE TORINESE,10851,5312,7075,3523,...,1262,PARTITO DEMOCRATICO - ITALIA DEMOCRATICA E PRO...,GARIGLIO,DAVIDE,TORINO,03/04/1967,M,1853,DAVIDE GARIGLIO,C-PIEMONTE 1-PIEMONTE 1 - P01-PIEMONTE 1 - U03...
4,25/9/2022 00:00:00,C,PIEMONTE 1,PIEMONTE 1 - P01,PIEMONTE 1 - U03 (COLLEGNO),CASELLE TORINESE,10851,5312,7075,3523,...,241,ALLEANZA VERDI E SINISTRA,GARIGLIO,DAVIDE,TORINO,03/04/1967,M,1853,DAVIDE GARIGLIO,C-PIEMONTE 1-PIEMONTE 1 - P01-PIEMONTE 1 - U03...


In [16]:
len(df["COLLPLURI"].unique())

49

In [4]:
df.groupby(["candidate", "DESCRLISTA"]).agg({"VOTICANDIDATO": "sum", "VOTILISTA": "sum"}).sort_values("VOTICANDIDATO")

VOTICANDIDATO  \
candidate         DESCRLISTA                                                 
ALESSANDRA FILICE FORZA DEL POPOLO                                      61   
LAURA CARERI      FORZA DEL POPOLO                                      78   
PATRIZIO MARALDI  MASTELLA NOI DI CENTRO EUROPEISTI                    106   
CARLO FIUMANO'    FORZA DEL POPOLO                                     109   
RITA  ROCCO       MASTELLA NOI DI CENTRO EUROPEISTI                    117   
...                                                                    ...   
CIRO MASCHIO      LEGA PER SALVINI PREMIER                          144942   
ALESSANDRO SORTE  FRATELLI D'ITALIA CON GIORGIA MELONI              153202   
                  FORZA ITALIA                                      153202   
                  NOI MODERATI/LUPI - TOTI - BRUGNARO - UDC         153202   
                  LEGA PER SALVINI PREMIER                          153202   

                                                             VOTILISTA  
candidate         DESCRLISTA                                            
ALESSANDRA FILICE FORZA DEL POPOLO                                  60  
LAURA CARERI      FORZA DEL POPOLO                                  74  
PATRIZIO MARALDI  MASTELLA NOI DI CENTRO EUROPEISTI                 96  
CARLO FIUMANO'    FORZA DEL POPOLO                                 103  
RITA  ROCCO       MASTELLA NOI DI CENTRO EUROPEISTI                103  
...                                                                ...  
CIRO MASCHIO      LEGA PER SALVINI PREMIER                       34467  
ALESSANDRO SORTE  FRATELLI D'ITALIA CON GIORGIA MELONI           79978  
                  FORZA ITALIA                                   21546  
                  NOI MODERATI/LUPI - TOTI - BRUGNARO - UDC       1620  
                  LEGA PER SALVINI PREMIER                       46219  

[2179 rows x 2 columns]

In [8]:
df_filtered = df.rename(columns={"VOTILISTA": "value", "DESCRLISTA": "party"})

tt = df_filtered.groupby(["polling_id", "candidate", "party"]).agg({"value": "sum"})
tt["rate"] = tt.groupby(level=[0], group_keys=False).apply(lambda x: x/x.sum())
tt = tt.reset_index()
tt = tt[["polling_id", "candidate", "party", "rate"]]

df_filtered = pd.merge(df_filtered[["polling_id", "candidate", "party", "value"]], tt, on=["polling_id", "candidate", "party"])
df_filtered.head()

,polling_id,candidate,party,value,rate
0,C-PIEMONTE 1-PIEMONTE 1 - P01-PIEMONTE 1 - U03...,ELENA MACCANTI,LEGA PER SALVINI PREMIER,631,0.098134
1,C-PIEMONTE 1-PIEMONTE 1 - P01-PIEMONTE 1 - U03...,ELENA MACCANTI,FORZA ITALIA,476,0.074028
2,C-PIEMONTE 1-PIEMONTE 1 - P01-PIEMONTE 1 - U03...,ELENA MACCANTI,NOI MODERATI/LUPI - TOTI - BRUGNARO - UDC,23,0.003577
3,C-PIEMONTE 1-PIEMONTE 1 - P01-PIEMONTE 1 - U03...,DAVIDE GARIGLIO,PARTITO DEMOCRATICO - ITALIA DEMOCRATICA E PRO...,1262,0.196267
4,C-PIEMONTE 1-PIEMONTE 1 - P01-PIEMONTE 1 - U03...,DAVIDE GARIGLIO,ALLEANZA VERDI E SINISTRA,241,0.037481


In [11]:
df_filtered.to_csv(f"data_output/Italy/{year}_general_commune.csv.gz", compression="gzip", index=False)

In [14]:
df_location = df[["polling_id", "CODTIPOELEZIONE", "CIRC-REG", "COLLPLURI", "COLLUNINOM", "COMUNE"]].drop_duplicates()

df_location.to_csv(f"data_output/Italy/{year}_general_commune_location.csv.gz", compression="gzip", index=False)